In [1]:
from ortools.linear_solver import pywraplp

In [2]:
def print_result(result: int, solver: pywraplp.Solver, 
                 delta: dict, I: set, J: set) -> None:
    
    if result == solver.OPTIMAL:
        print("Optimal solution found.")
    
    obj = solver.Objective().Value()
    print(f"obj = {round(obj, 2)}")
    
    I = list(I)
    I.sort()
    
    for i in list(I):
        for j in J:
            delta = delta_ij[i][j]
            
            if delta_ij[i][j].SolutionValue() == 1:
                print(f"Department {i} is located in {j}")
            else:
                pass
    
    return None

## Problems

These problems are designed to show some typical MILP problems. The problem focuses on a typical business problem called decentralisation, and is taken from the textbook [Model Building in Mathematical Programming by H.P. Williams](https://www.wiley.com/en-gb/Model+Building+in+Mathematical+Programming%2C+5th+Edition-p-9781118443330). Chapter 12 of that textbook has many more interesing MILP problems if people want to do any further reading. 

The problem has been broken into two parts. The first is a relatively simple MILP about the location of various facilities. The second adds an additional layer of complexity to the problem, and highlights one of the key transforms we can do when introducing integers into the problem.

When completing these exercises, remember some of the key steps to solving a mathematical program:

1. Describe the **constraints** and **objectives** verbally.
2. Describe the **entities** as sets.
3. Describe any **parameters** as sets.
4. Define the **decision variables** that seem sufficient to solve the problem.
5. Write the **objective function** using the parameters and decision variables.
6. Write the **constraints** using sets and decision variables.
7. Iterate & repeat until happy.

### Decentralisation 1

A large company wishes to move some of its departments out of London. There are benefits to be derived from doing this (cheaper housing, government incentives, easier recruitment, etc.), which have been costed.

The company comprises five departments (A, B, C, D and E). The possible cities for relocation are Bristol and Brighton, or a department may be kept in London. None of these cities (including London) may be the location for more than three of the departments.

Benefits to be derived from each relocation are given (in thousands of pounds per year) as follows:

| City     | A  | B  | C  | D  | E  |
|----------|----|----|----|----|----|
| Bristol  | 10 | 15 | 10 | 20 | 5  |
| Brighton | 10 | 20 | 15 | 15 | 15 |

In [3]:
solver = pywraplp.Solver("decentralisation", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

### 1. Describe the rules and constraints verbally

- Departments A, B, C, D and E can be located in one of three locations.
- There is a cost benefit for locating each department in a specific city.
- No city can have more than three departments.
- No department can be split across multiple cities.
- Objective is to distribute departments between locations in order to maximise cost benefit. 

### 2. Describe the entities as sets

$J \in \{A, B, C, D, E \}$ [departments]

$I \in \{\mathrm{brighton}, \mathrm{bristol}, \mathrm{london} \}$ [cities]

In [4]:
I = {"A", "B", "C", "D", "E"}
J = {"bristol", "brighton", "london"}

### 3. Describe related parameters as sets

These are the associated benefits. They can be easily described as a matrix of $i$ and $j$ pairs. This time, we also have two additional matrices to describe the between-department volumes of communication and the communication costs between cities. 

In [5]:
# benefits matrix
V_ij = {"A": {"bristol": 10, "brighton": 10, "london": 0},
        "B": {"bristol": 15, "brighton": 20, "london": 0}, 
        "C": {"bristol": 10, "brighton": 15, "london": 0}, 
        "D": {"bristol": 20, "brighton": 15, "london": 0}, 
        "E": {"bristol": 5, "brighton": 15, "london": 0}}

### 4. Decision variables

In this first problem, the only decision variable we need to know about is where to locate each department. These can be described by binary decision variables:

$\delta_{ij} \in \{0, 1\} \, \, \forall i \in I, j \in J$

In [6]:
# where to locate each department
delta_ij = {i: {j: solver.IntVar(lb=0, ub=1, name=f"delta_{i}{j}") for j in J} for i in I}

### 5. Objective function

Objective is simply to maximise the value we make from moving the departments. The value is only generated iff the department is located in the city.

$\max(\sum_{i \in I, j \in J} \delta_{ij} \cdot V_{ij})$ 

In [7]:
solver.Maximize(solver.Sum(delta_ij[i][j] * V_ij[i][j] for i in I for j in J))

### 6. Constraints

$\sum_{i \in I}l_{ij} \leq 3 \, \, \forall \, \, j \in J$ [can only have 3 departments per location]

$\sum_{j \in J}l_{ij} = 1 \, \, \forall \, \, i \in I$ [each department can only be in 1 location]

In [8]:
for j in J:
    solver.Add(solver.Sum(delta_ij[i][j] for i in I) <= 3)
    
for i in I:
    solver.Add(solver.Sum(delta_ij[i][j] for j in J) == 1)

In [9]:
# run solver
result = solver.Solve()
print_result(result=result, solver=solver, delta=delta_ij, I=I, J=J)

Optimal solution found.
obj = 80.0
Department A is located in bristol
Department B is located in brighton
Department C is located in brighton
Department D is located in bristol
Department E is located in brighton


### Decentralisation 2

It has been suggested that there will be greater costs of communication between departments cause by relocation across multiple sites. These have also been costed for all possible locations of each department.

Communication costs are of the form $C_{ik} \cdot D_{jl}$, where $C_{ik}$ is the quantity of communication between departments $i$ and $k$ per year and $D_{jl}$ is the cost per unit of communication between cities $j$ and $l$. $C_{ik}$ and $D_{jl}$ are given by the following tables:

|   | A | B   | C   | D   | E   |
|---|---|-----|-----|-----|-----|
| A |   | 0.0 | 1.0 | 1.5 | 0.0 |
| B |   |     | 1.4 | 1.2 | 0.0 |
| C |   |     |     | 0.0 | 2.0 |
| D |   |     |     |     | 0.7 |

|          | Bristol | Brighton | London |
|----------|---------|----------|--------|
| Bristol  |  5.0    | 14.0     | 13.0   |
| Brighton |         |   5.0    | 9.0    |
| London   |         |          |  10.0  |

Where should each department be located so as to minimise overall yearly cost?

#### Hint:

If we want to have a binary decision variable, $\gamma$ that is 1 iff $\delta_1 = 1$ and $\delta_2 = 1$, we can introduce the following constraints:

$\gamma - \delta_1 \leq 0$

$\gamma - \delta_2 \leq 0$

$\delta_1 + \delta_2 - \gamma \leq 1$

In [10]:
solver = pywraplp.Solver("decentralisation-2", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

### 1. Describe the rules and constraints verbally

- Departments A, B, C, D and E can be located in one of three locations.
- There is a cost benefit for locating each department in a specific city.
- No city can have more than three departments.
- No department can be split across multiple cities.
- Objective is to distribute departments between locations in order to maximise cost benefit. 
- **There is a communication cost incurred for locating different departments in different cities. This depends on the department and location.**

### 2. Describe the entities as sets

$J \in \{A, B, C, D, E \}$ [departments]

$I \in \{\mathrm{brighton}, \mathrm{bristol}, \mathrm{london} \}$ [cities]

In [11]:
I = {"A", "B", "C", "D", "E"}
J = {"bristol", "brighton", "london"}

### 3. Describe related parameters as sets

These are the associated benefits. They can be easily described as a matrix of $i$ and $j$ pairs. This time, we also have two additional matrices to describe the between-department volumes of communication and the communication costs between cities. 

In [12]:
# benefits matrix
V_ij = {"A": {"bristol": 10, "brighton": 10, "london": 0},
        "B": {"bristol": 15, "brighton": 20, "london": 0}, 
        "C": {"bristol": 10, "brighton": 15, "london": 0}, 
        "D": {"bristol": 20, "brighton": 15, "london": 0}, 
        "E": {"bristol": 5, "brighton": 15, "london": 0}}

# communication volume matrix
C_ik = {"A": {"A": 0.0, "B": 0.0, "C": 1.0, "D": 1.5, "E": 0.0},
        "B": {"A": 0.0, "B": 0.0, "C": 1.4, "D": 1.2, "E": 0.0},
        "C": {"A": 1.0, "B": 1.4, "C": 0.0, "D": 0.0, "E": 2.0},
        "D": {"A": 1.5, "B": 1.2, "C": 0.0, "D": 0.0, "E": 0.7},
        "E": {"A": 0.0, "B": 0.0, "C": 2.0, "D": 0.7, "E": 0.0}}

# cost matrix
D_jl = {"bristol": {"bristol": 5.0, "brighton": 14.0, "london": 13.0},
        "brighton": {"bristol": 14.0, "brighton": 5.0, "london": 9.0},
        "london": {"bristol": 13.0, "brighton": 9.0, "london": 10.0}}

### 4. Decision variables

In this first problem, the only decision variable we need to know about is where to locate each department. These can be described by binary decision variables:

$\delta_{ij} \in \{0, 1\} \, \, \forall i \in I, j \in J$

**We will also need a second variable in the objective to define pairs of departments. This is because we cannot multiply two binary variables in the objective value, so we need to define another variable $\gamma_{ijkl}$ that is 1 iff department $i$ is in location $j$ and department $k$ is in location $l$.**

$\gamma_{ijkl} \in \{0, 1 \} \, \, \forall i \in I, j \in J, k \in I, l \in J$

In [13]:
# where to locate each department
delta_ij = {i: {j: solver.IntVar(lb=0, ub=1, name=f"delta_{i}{j}") for j in J} for i in I}

# tracks pairs of departments
gamma_ijkl = {i: {j: {k: {l: solver.IntVar(lb=0, ub=1, name=f"gamma_{i}{j}{k}{l}") 
              for l in J} for k in I} for j in J} for i in I}

### 5. Objective function

Objective is simply to maximise the value we make from moving the departments. The value is only generated iff the department is located in the city. **We must also include a communication overhead term to account for the cases where $\gamma_{ijkl} = 1$**:

$\max(\sum_{i \in I, j \in J} \delta_{ij} \cdot V_{ij} - 0.5 \cdot \sum_{i \in I, k \in I, j \in J, l \in J} C_{ik} \cdot D_{jl} \cdot \gamma_{ijkl})$

In [14]:
solver.Maximize(solver.Sum(delta_ij[i][j] * V_ij[i][j] for i in I for j in J)
                - 0.5 * solver.Sum(C_ik[i][k] * D_jl[j][l] * gamma_ijkl[i][j][k][l]
                                   for i in I for j in J for k in I for l in J))

### 6. Constraints

$\sum_{i \in I}l_{ij} \leq 3 \, \, \forall \, \, j \in J$ [can only have 3 departments per location]

$\sum_{j \in J}l_{ij} = 1 \, \, \forall \, \, i \in I$ [each department can only be in 1 location]

**We also need to introduce another set of constraints that ensure that $\gamma_{ijkl}$ has the correct values. This can be introduced by using the trick in the problem statement.**

#### $\gamma$ constraints

$\gamma_{ijkl} - \delta_{ij} \leq 0 \, \, \forall i \in I, j \in J, k \in I, l \in J$

$\gamma_{ijkl} - \delta_{kl} \leq 0 \, \, \forall i \in I, j \in J, k \in I, l \in J$

$\delta_{ij} + \delta_{kl} - \gamma_{ijkl} \leq 1 \, \, \forall i \in I, j \in J, k \in I, l \in J$

In [15]:
for j in J:
    solver.Add(solver.Sum(delta_ij[i][j] for i in I) <= 3)
    
for i in I:
    solver.Add(solver.Sum(delta_ij[i][j] for j in J) == 1)

for i in I:
    for j in J:
        for k in I:
            for l in J:
                solver.Add(gamma_ijkl[i][j][k][l] - delta_ij[i][j] <= 0)
                solver.Add(gamma_ijkl[i][j][k][l] - delta_ij[k][l] <= 0)
                solver.Add(delta_ij[i][j] + delta_ij[k][l] - gamma_ijkl[i][j][k][l] <= 1)

In [16]:
# run solver
result = solver.Solve()
print_result(result=result, solver=solver, delta=delta_ij, I=I, J=J)

Optimal solution found.
obj = 14.9
Department A is located in bristol
Department B is located in brighton
Department C is located in brighton
Department D is located in bristol
Department E is located in brighton
